In [1]:
from __future__ import print_function, division
%matplotlib inline

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable,Function
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from scipy.optimize import linear_sum_assignment

import sys 
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import os
plt.ion()   # interactive mode
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")


# Any results you write to the current directory are saved as output.

In [2]:
TRAIN_PATH = '../DATA/stage1_train/'
TEST_PATH = '../DATA/stage1_test/'
UNET_PATH = '../DATA/unetdata'

# Loss Function

In [3]:
class loss_f(nn.Module):
    def __init__(self,lamda=1):
        # super parameter
        super(loss_f,self).__init__()
        self.lamda = lamda
    def forward(self,output_list,scores,gt_masks):
        """
        gt_masks: sample['masks'],shape=[1,masks_number,H,W]
        output_list: list length=[premasks_number],element shape = [H,W]
        scores: list length = [premasks_numbers]
        batchsize = 1
        """
        gt_masks = gt_masks.squeeze(0) # max_element = 1
        num_gt = gt_masks.size()[0]
        num_pre = len(output_list) 
        num_min = min(num_gt,num_pre)
        real_output = torch.stack(output_list) # shape = [premasks,H,W]
        rps = torch.stack(scores)
        loss = 0
        Matrix = []
        for i in range(num_min): # when pre>gt, drop the later prediction
            for j in range(num_gt): 
                #generate a num_gt*num_pre matrix 
                IoU = self._iou(real_output[i],gt_masks[j]) ## IoU is a tensor scalar
                Matrix.append(IoU)

        Matrix = torch.stack(Matrix).view(num_min,num_gt)
        M = hungarian()(Matrix)
#         print('-M',torch.sum(-M).item())
#         print('rps=0',rps[0])
#         print('rps=num_min',rps[num_min-1])
        for i in range(num_min):
            loss = loss + (M[i]+self.lamda*F.binary_cross_entropy(rps[i],torch.tensor([1.0]).to(device)))
        for i in range(num_gt,num_pre):
            loss = loss + self.lamda*F.binary_cross_entropy(rps[i],torch.tensor([0.0]).to(device)) #F.binary_cross_entropy(b,a)

        return loss
    def _iou(self,x,y):
        iou_inter = torch.sum(torch.mul(x,y))
        iou = iou_inter/(torch.sum(x)+torch.sum(y)-iou_inter)
        return -iou
    
class hungarian(Function):

    def forward(ctx,input):
        numpy_input = input.cpu().detach().numpy()
        matrix = np.zeros_like(numpy_input)
        row_ind,col_ind = linear_sum_assignment(numpy_input)
        length = len(row_ind)
        M = []
        for i in range(length):
            matrix[row_ind[i]][col_ind[i]] = numpy_input[row_ind[i]][col_ind[i]]
            M.append(numpy_input[row_ind[i]][col_ind[i]])
        M = np.stack(M)
        matrix = torch.from_numpy(matrix).cuda()
        ctx.save_for_backward(matrix)
        result = torch.from_numpy(M).cuda()
        
        return result

    def backward(ctx,grad_output):
        
        matrix,=ctx.saved_tensors
        grad_input = matrix.clone()
        return grad_input

#         result = torch.from_numpy(matrix).type(torch.FloatTensor).cuda()
#         return result

# Model Class

In [4]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x64 = self.up4(x, x1)
        x = self.outc(x64)
        return (F.sigmoid(x),x64)

# sub-parts of the U-Net model

class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        
        # for padding issues, see 
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd

        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()       
        # ConvLSTM part
        self.conv = nn.Conv2d(64,16,3,stride=1,padding=1)
        self.convlstm = ConvLSTM(input_size=(176,176),
                                 input_dim=16,
                                 hidden_dim=[16],
                                 kernel_size=(3, 3),
                                 num_layers=1,
                                 batch_first=False,
                                 bias=True,
                                 return_all_layers=True)
        # SI part
        
        self.conv8 = nn.Conv2d(16,1,1,stride=1,padding=0)
        self.fc = nn.Linear(16*176*176, 1)
        
    def forward(self,x,score_bound,seq_len):
        # x.shape() = [1,64,176,176]
        x = self.conv(x)#[1,16,176,176]
        size = x.size()[-1] # W
        # convlstm and si
        convlstm_input = x.unsqueeze(0) #[1,batch,d,h,w],(t, b, c, h, w) -> (b, t, c, h, w) (i.e.[1,1,16,176,176])
        output_list = []
        scores = []
        hidden_state=None
        for i in range(seq_len):
            [layer_output_list, last_state_list] = self.convlstm(convlstm_input,hidden_state) # layer_output_list=[1,batch,t,d,h,w],t=1,(i.e.[1,1,16,176,176])
                                                                                 # last_state_list(i.e. [[h,c]])=[1,2,batch,t,d,h,w] (i.e.[1,2,1,1,16,176,176])
            hidden_state = last_state_list

            layer_output_list = layer_output_list[0].squeeze()
            
            # produce score
            score_input = layer_output_list.unsqueeze(0)
            #score_input = F.max_pool2d(score_input,2) #[88,88]
            [b,c,h,w] = score_input.size()
            score = F.sigmoid(self.fc(score_input.view(b*c*h*w)))
            scores.append(score)
            # produce masks
            SI = F.sigmoid(
                    F.log_softmax(
                        self.conv8(layer_output_list.unsqueeze(0)))) #the input is [1,d,h,w],SI is [h,w] (i.e.[16,176,176])

            mask = SI.squeeze() #[batch,h,w] ,since batchsize=1,it should be [h,w] (i.e.[176,176])
            output_list.append(mask)
        return (output_list,scores)

# Helper Classes

In [6]:
class NeuralDataset(Dataset):
    """Neural dataset."""

    def __init__(self,root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the examples.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.example_list = os.listdir(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.example_list)

    def __getitem__(self, idx):
        example_dir = os.path.join(self.root_dir,
                                self.example_list[idx])
        img_dir = example_dir+'/images'
        masks_dir = example_dir+'/masks'
        
        img_name = img_dir+'/'+os.listdir(img_dir)[0]
        image = io.imread(img_name)[:,:,0:3]
        
        maskwalk = os.walk(masks_dir).__next__()
        masks = []
        for item in maskwalk[2]:
            masks_name = os.path.join(masks_dir,item)
            mask = io.imread(masks_name)
            masks.append(mask)
        masks = np.stack(masks)
        
        sample = {'image': image, 'masks': masks}# masks is [masknumber,H,W],image [H,W,C]

        if self.transform:
            sample = self.transform(sample)

        return sample
    
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # resize the masks
        new_msk = []
        for mask in masks:
            new_msk.append(transform.resize(mask, (new_h, new_w)))
        new_mask = np.stack(new_msk)
        
        return {'image': img, 'masks': new_mask}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']

        h, w = image.shape[:2]
        
        new_h, new_w = self.output_size
        if h-new_h==0:
            top = 0
        else:
            top = np.random.randint(0, h - new_h)
        if w-new_w==0:
            left = 0
        else:
            left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        # crop the masks
        new_msk = []
        for mask in masks:
            newm = mask[top: top + new_h,
                        left: left + new_w]
            new_msk.append(newm)
        new_mask = np.stack(new_msk)

        return {'image': image, 'masks': new_mask}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, masks = sample['image'], sample['masks']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        # numpy masks: N x H x W
        # torch masks: N X H X W
        return {'image': torch.from_numpy(image),
                'masks': torch.from_numpy(masks)}
class ConvLSTMCell(nn.Module):

    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        
        Parameters
        ----------
        input_size: (int, int)
            Height and width of input tensor as (height, width).
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.height, self.width = input_size
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding     = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias        = bias
        
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        # input_tensor is [batch,channel,h,w]
        h_cur, c_cur = cur_state     
        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis
        
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1) 
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next) # size of h and c is [batch,channel,h,w]
        
        return h_next, c_next

    def init_hidden(self, batch_size):
        return [Variable(torch.zeros(batch_size, self.hidden_dim, self.height, self.width)).cuda(),
                Variable(torch.zeros(batch_size, self.hidden_dim, self.height, self.width)).cuda()]


class ConvLSTM(nn.Module):

    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim  = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.height, self.width = input_size

        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i-1]

            cell_list.append(ConvLSTMCell(input_size=(self.height, self.width),
                                          input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)
        

    def forward(self, input_tensor, hidden_state=None):
        """
        
        Parameters
        ----------
        input_tensor: todo 
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
            
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        # Implement stateful ConvLSTM
        if hidden_state is None:
            hidden_state = self._init_hidden(batch_size=input_tensor.size(0))

        layer_output_list = []
        last_state_list   = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):  #num_layers should be 2

            h, c = hidden_state[layer_idx]
            output_inner = []
            
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])# size of h and c is [batch,c,h,w]
                output_inner.append(h)#[t,batch,c,h,w]

            layer_output = torch.stack(output_inner, dim=1)# layer_output is [batch,t,c,h,w]
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)# [1,batch,t,c,h,w]
            last_state_list.append([h, c])#[1,2,batch,t,c,h,w]
        torch.cuda.empty_cache()
        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list   = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size))
        return init_states # [[h,c]]

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                    (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [7]:
# reading data
transformed_dataset = NeuralDataset(root_dir=TRAIN_PATH,
                                    transform=transforms.Compose([
                                        Rescale(176),
                                        RandomCrop(176),
                                        ToTensor()]))

dataloader = DataLoader(transformed_dataset, batch_size=1,
                        shuffle=True, num_workers=0)

In [8]:
#set up model
unet = torch.load('unet.pkl')
net = Net().to(device)
# training
torch.cuda.empty_cache()
# create your optimizer
criterion = loss_f()
optimizer = optim.SGD(net.parameters(), lr=0.01,)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [9]:
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs
        inputs = (data['image'].type(torch.float32)).to(device)
        labels = data['masks'].type(torch.float32).to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        seq_len = 2
        with torch.no_grad():
            (x,x64) = unet(inputs)
        (output_list,scores) = net(x64,0.5,seq_len)
        loss = criterion(output_list,scores,labels) #  the minimum of loss is -1
        loss.backward()
        torch.nn.utils.clip_grad_norm_(filter(lambda p:p.requires_grad,net.parameters()), max_norm=5, norm_type=1)
        optimizer.step()
        scheduler.step(loss)
        
        #print('epoch=',epoch,'num_data=',i,'seq_len=',seq_len,'loss',loss.item(),'lr',optimizer.param_groups[0]['lr'])
        # print statistics
        running_loss += loss.item()
        if i % 40 == 39:    # print every 134 mini-batches
            print('[%d, %5d] loss: %.3f,seq_len:%d' %
                  (epoch + 1, i + 1, running_loss /134,seq_len))
            running_loss = 0.0
#             torch.save(net.state_dict(), '../mini_program/net_params.pkl')
        
    if seq_len>=420:
        break

print('Finished Training')

epoch= 0 num_data= 0 seq_len= 2 loss 1.3240092992782593 lr 0.01
epoch= 0 num_data= 1 seq_len= 2 loss 1.3303252458572388 lr 0.01
epoch= 0 num_data= 2 seq_len= 2 loss 1.3340572118759155 lr 0.01
epoch= 0 num_data= 3 seq_len= 2 loss 1.3330788612365723 lr 0.01
epoch= 0 num_data= 4 seq_len= 2 loss 1.2698934078216553 lr 0.01
epoch= 0 num_data= 5 seq_len= 2 loss 1.3371150493621826 lr 0.01
epoch= 0 num_data= 6 seq_len= 2 loss 1.3181287050247192 lr 0.01
epoch= 0 num_data= 7 seq_len= 2 loss 1.3149006366729736 lr 0.01
epoch= 0 num_data= 8 seq_len= 2 loss 1.3416309356689453 lr 0.01
epoch= 0 num_data= 9 seq_len= 2 loss 1.3403486013412476 lr 0.01
epoch= 0 num_data= 10 seq_len= 2 loss 1.2429008483886719 lr 0.01
epoch= 0 num_data= 11 seq_len= 2 loss 1.3313279151916504 lr 0.01
epoch= 0 num_data= 12 seq_len= 2 loss 1.2857414484024048 lr 0.01
epoch= 0 num_data= 13 seq_len= 2 loss 1.307157278060913 lr 0.01
epoch= 0 num_data= 14 seq_len= 2 loss 1.2794618606567383 lr 0.01
epoch= 0 num_data= 15 seq_len= 2 los

KeyboardInterrupt: 